In [1]:
from canonical_toolkit.base.matrix import MatrixInstance, MatrixSeries, MatrixFrame

import numpy as np
import scipy.sparse as sp
from pathlib import Path
import tempfile
import os

### MATRIX

In [2]:
print("=== 1. Initialization & Representation ===")
# Dense Matrix
dense_data = np.array([[1.0, 2.0], [3.0, 4.0]])
mat_dense = MatrixInstance(
    matrix=dense_data, 
    label="dense_test", 
    tags={"type": "raw", "quality": 100}
)
print(f"Dense Instance:\n{mat_dense}")
assert mat_dense.shape == (2, 2)
assert mat_dense.label == "dense_test"

=== 1. Initialization & Representation ===


MatrixInstance:          
dense_test               
┏━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Property ┃ Value      ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━┩
│ Shape    │ [2×2]      │
│ Storage  │ Dense      │
│ Type     │ raw        │
│ Quality  │ 100        │
│ Values   │ 1.00  2.00 │
│          │ 3.00  4.00 │
└──────────┴────────────┘

Dense Instance:



In [3]:
# Sparse Matrix
sparse_data = sp.csr_matrix([[0, 0, 1], [1, 0, 0], [0, 2, 0]])
mat_sparse = MatrixInstance(
    matrix=sparse_data, 
    label="sparse_test", 
    tags={"sparsity": 0.66}
)
print(f"\nSparse Instance:\n{mat_sparse}")
assert sp.issparse(mat_sparse.matrix)

MatrixInstance: sparse_test    
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Property ┃ Value            ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Shape    │ [3×3]            │
│ Storage  │ Sparse (csr)     │
│ Non-zero │ 3 / 9            │
│ Sparsity │ 66.67%           │
│ Sparsity │ 0.66             │
│ Samples  │   [0,2] = 1.0000 │
│          │   [1,0] = 1.0000 │
│          │   [2,1] = 2.0000 │
└──────────┴──────────────────┘


Sparse Instance:



In [4]:
# Zero Factory
mat_zero = MatrixInstance.zeros(
    shape=(3, 3), 
    label="gap_filler", 
    sparse=True, 
    is_gap=True
)
print(f"\nZero Instance (Factory):\n{mat_zero}")
assert mat_zero.tags["is_gap"] is True

MatrixInstance: gap_filler 
┏━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Property ┃ Value        ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Shape    │ [3×3]        │
│ Storage  │ Sparse (csr) │
│ Non-zero │ 0 / 9        │
│ Sparsity │ 100.00%      │
│ Is_gap   │ True         │
└──────────┴──────────────┘


Zero Instance (Factory):



In [5]:
print("\n=== 2. Math Operations ===")
# Addition (Dense)
mat_a = MatrixInstance(np.array([10]), "A")
mat_b = MatrixInstance(np.array([20]), "B")
mat_sum = mat_a + mat_b
print(f"10 + 20 = {mat_sum.matrix[0]}")
assert mat_sum.matrix[0] == 30
# Note: Metadata usually comes from the left operand ("A")
assert mat_sum.label == "A" 


=== 2. Math Operations ===
10 + 20 = 30


In [6]:
# Operations with Scalars
mat_scaled = mat_dense * 2
print(f"Dense * 2:\n{mat_scaled.matrix}")
assert mat_scaled.matrix[0, 1] == 4.0

Dense * 2:
[[2.000 4.000]
 [6.000 8.000]]


In [7]:
# Operations with Sparse
# Note: Adding dense scalar to sparse matrix usually densifies it or fails depending on implementation.
# Adding two sparse matrices should work.
mat_sp_sum = mat_sparse + mat_sparse
print(f"Sparse + Sparse (Element [0,2]): {mat_sp_sum.matrix[0, 2]}")
assert mat_sp_sum.matrix[0, 2] == 2

Sparse + Sparse (Element [0,2]): 2


In [8]:
print("\n=== 3. I/O (Save & Load) ===")
with tempfile.TemporaryDirectory() as tmpdir:
    save_path = Path(tmpdir) / "test_matrix"
    
    # Save
    mat_dense.save(save_path, overwrite=True)
    print(f"Saved to: {save_path}.json and .npy")
    
    # Load
    loaded_mat = MatrixInstance.load(save_path)
    print(f"Loaded: {loaded_mat.label}")
    
    # Verify
    np.testing.assert_array_equal(loaded_mat.matrix, mat_dense.matrix)
    assert loaded_mat.tags == mat_dense.tags
    print("Save/Load verification passed.")
    
    # Test Subset Loading
    # Load only top-left 1x1
    subset_mat = MatrixInstance.load(save_path, subset_indices=[0])
    print(f"Subset Shape: {subset_mat.shape}")
    assert subset_mat.shape == (1, 1)
    assert subset_mat.matrix[0, 0] == 1.0
    print("Subset loading passed.")


=== 3. I/O (Save & Load) ===
Saved to: /var/folders/1r/pgxzj8b13d98r7y7z49g3dp40000gn/T/tmpkuc885w7/test_matrix.json and .npy
Loaded: dense_test
Save/Load verification passed.
Subset Shape: (1, 1)
Subset loading passed.


### Series

In [9]:
print("=== 1. Initialization & Gap Filling ===")
# Create two instances
m1 = MatrixInstance(np.eye(2), label="test_series", tags={"radius": 0})
m3 = MatrixInstance(np.eye(2) * 3, label="test_series", tags={"radius": 2})

# Initialize with a 'gap' (None) at index 1
# Input: [Instance(0), None, Instance(2)]
series = MatrixSeries(matrices=[m1, None, m3])

print(f"Series Representation:\n{series}")

assert len(series.matrices) == 3
assert series.indices == [0, 1, 2]
assert series[0].matrix[0, 0] == 1.0
assert series[2].matrix[0, 0] == 3.0

# Verify gap filling at index 1
# It should be a zero matrix of the same shape/type as others
assert np.all(series[1].matrix == 0)
assert series[1].label == "test_series"
print("Gap filling passed.")

=== 1. Initialization & Gap Filling ===


MatrixSeries: test_series                            
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Index ┃                test_series                ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     0 │         [2×2] Dense {'radius': 0}         │
│     1 │ [2×2] Dense {'radius': 1, 'is_gap': True} │
│     2 │         [2×2] Dense {'radius': 2}         │
└───────┴───────────────────────────────────────────┘

Series Representation:

Gap filling passed.


In [10]:
print("\n=== 2. Label Overwriting ===")
# Instances with different labels
ma = MatrixInstance(np.eye(2), label="Label_A")
mb = MatrixInstance(np.eye(2), label="Label_B")

# This should raise ValueError because labels differ and no override provided
try:
    MatrixSeries(matrices=[ma, mb])
except ValueError as e:
    print(f"Caught expected inconsistency error: {e}")

# This should work because we provide a master label to overwrite
series_overwritten = MatrixSeries(matrices=[ma, mb], label="Master_Label")
assert series_overwritten.label == "Master_Label"
assert series_overwritten[0].label == "Master_Label"
print("Label overwriting passed.")



=== 2. Label Overwriting ===
Caught expected inconsistency error: Inconsistent labels: {'Label_B', 'Label_A'}. Provide a label to overwrite.
Label overwriting passed.


In [11]:
print("\n=== 3. Slicing & Indexing ===")
# Create a longer series
long_series = MatrixSeries(matrices=[m1, None, m3, None, m1])
print(f"Long Series indices: {long_series.indices}")

# Slicing returns a new Series
sub_series = long_series[1:4]
print(f"Sliced Series (1:4) indices: {sub_series.indices}")
assert len(sub_series.matrices) == 3
assert isinstance(sub_series, MatrixSeries)

# Negative indexing
assert long_series[-1].matrix[0, 0] == 1.0
print("Slicing and indexing passed.")


=== 3. Slicing & Indexing ===
Long Series indices: [0, 1, 2, 3, 4]
Sliced Series (1:4) indices: [0, 1, 2]
Slicing and indexing passed.


In [12]:
print("\n=== 4. Math & Transformations ===")
# Addition of two series
s1 = MatrixSeries(matrices=[m1, m1])
s2 = MatrixSeries(matrices=[m1, m1])
s_sum = s1 + s2
assert s_sum[0].matrix[0, 0] == 2.0
print("Series addition passed.")

# Map transformation
# Double all matrices in the series
s_doubled = s1.map(lambda m: m * 2, inplace=False)
assert s_doubled[0].matrix[0, 0] == 2.0
assert s1[0].matrix[0, 0] == 1.0 # Original preserved
print("Series mapping passed.")


=== 4. Math & Transformations ===
Series addition passed.
Series mapping passed.


In [13]:
print("\n=== 5. I/O (Save & Load) ===")
with tempfile.TemporaryDirectory() as tmpdir:
    save_dir = Path(tmpdir) / "test_series_folder"
    
    # Save series
    series.save(folder_path=tmpdir, series_name="test_series_folder", suffix_number=False)
    print(f"Series saved to {save_dir}")
    
    # Load series
    loaded_series = MatrixSeries.load(save_dir)
    print(f"Loaded series: {loaded_series.label} with {len(loaded_series.matrices)} matrices")
    
    assert loaded_series.label == series.label
    assert len(loaded_series.matrices) == len(series.matrices)
    np.testing.assert_array_equal(loaded_series[2].matrix, series[2].matrix)
    print("Save/Load verification passed.")

print("\n=== All Series Tests Passed! ===")


=== 5. I/O (Save & Load) ===
Series saved to /var/folders/1r/pgxzj8b13d98r7y7z49g3dp40000gn/T/tmpm_5ghwks/test_series_folder
Loaded series: test_series with 3 matrices
Save/Load verification passed.

=== All Series Tests Passed! ===


### Frame

In [14]:
print("=== 1. Initialization & Alignment ===")
# Create two series of different lengths
m = MatrixInstance(np.eye(2), label="template")

# Series A: Length 2
s_a = MatrixSeries(matrices=[m, m], label="FRONT")
# Series B: Length 4
s_b = MatrixSeries(matrices=[m, m, m, m], label="BACK")

# Initialize Frame - This should automatically trigger alignment (padding FRONT to 4)
frame = MatrixFrame(series=[s_a, s_b])

print(f"Frame Representation:\n{frame}")

assert len(frame.labels) == 2
assert len(frame[:, 0].matrices) == 4 # FRONT should now have 4 matrices
assert len(frame[:, 1].matrices) == 4 # BACK has 4
assert np.all(frame[:, 0].matrices[3].matrix == 0) # FRONT[3] should be zero-padded
print("Automatic alignment passed.")


=== 1. Initialization & Alignment ===


   MatrixFrame (2 series × 4   
           indices)            
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Index ┃  FRONT   ┃   BACK   ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│     0 │ [2×2] Dn │ [2×2] Dn │
│     1 │ [2×2] Dn │ [2×2] Dn │
│     2 │ [2×2] Dn │ [2×2] Dn │
│     3 │ [2×2] Dn │ [2×2] Dn │
└───────┴──────────┴──────────┘

Frame Representation:

Automatic alignment passed.


In [15]:
print("\n=== 2. Multi-Dimensional Indexing ===")
# Accessing a single series (column)
col_front = frame["FRONT"]
assert isinstance(col_front, MatrixSeries)
assert col_front.label == "FRONT"

# Accessing a single radius across all series (row)
# frame[radius_index] -> Returns Frame with 1 matrix per series
row_2 = frame[2]
assert isinstance(row_2, MatrixFrame)
assert len(row_2.series) == 2
assert len(row_2.series[0].matrices) == 1
print("Row/Column indexing passed.")

# 2D Tuple Indexing: frame[radius, series]
# Access Radius 1 of Series "BACK"
mat_1_back = frame[1, "BACK"]
assert isinstance(mat_1_back, MatrixInstance)
assert mat_1_back.label == "BACK"

# Sliced 2D Access: frame[0:2, ["FRONT", "BACK"]]
sub_frame = frame[0:2, ["FRONT", "BACK"]]
assert sub_frame.labels == ["FRONT", "BACK"]
assert len(sub_frame.series[0].matrices) == 2
print("2D Tuple indexing passed.")


=== 2. Multi-Dimensional Indexing ===
Row/Column indexing passed.
2D Tuple indexing passed.


In [16]:
print("\n=== 3. Transformation & Map ===")
# Map a function over all series
# Example: Scale all matrices in all series by 5
scaled_frame = frame.map(lambda s: s.map(lambda m: m * 5, inplace=False), inplace=False)
assert scaled_frame["FRONT"][0].matrix[0, 0] == 5.0
assert frame["FRONT"][0].matrix[0, 0] == 1.0 # Original preserved
print("Frame mapping passed.")


=== 3. Transformation & Map ===
Frame mapping passed.


In [17]:
print("\n=== 4. I/O (Hierarchical Save & Load) ===")
with tempfile.TemporaryDirectory() as tmpdir:
    save_dir = Path(tmpdir) / "test_frame_folder"
    
    # Save Frame
    frame.save(folder_path=tmpdir, frame_name="test_frame_folder", suffix_number=False)
    print(f"Frame saved to {save_dir}")
    
    # Check structure (hierarchical)
    assert (save_dir / "frame_meta.json").exists()
    # Check that each series has its own folder (usually named by description)
    # The folders are inside the frame folder
    print(f"Contents of {save_dir}: {os.listdir(save_dir)}")
    
    # Load Frame
    loaded_frame = MatrixFrame.load(save_dir)
    print(f"Loaded Frame: {len(loaded_frame.labels)} series, {len(loaded_frame.series[0].matrices)} indices")
    
    assert loaded_frame.labels == frame.labels
    assert len(loaded_frame.series[0].matrices) == len(frame.series[0].matrices)
    np.testing.assert_array_equal(loaded_frame[0, 0].matrix, frame[0, 0].matrix)
    print("Hierarchical Save/Load verification passed.")

print("\n=== All Frame Tests Passed! ===")


=== 4. I/O (Hierarchical Save & Load) ===
Frame saved to /var/folders/1r/pgxzj8b13d98r7y7z49g3dp40000gn/T/tmpipn3f5k6/test_frame_folder
Contents of /var/folders/1r/pgxzj8b13d98r7y7z49g3dp40000gn/T/tmpipn3f5k6/test_frame_folder: ['matrixseries_front_4i', 'frame_meta.json', 'matrixseries_back_4i']
Loaded Frame: 2 series, 4 indices
Hierarchical Save/Load verification passed.

=== All Frame Tests Passed! ===


In [18]:
print("=== MatrixFrame Hybrid Indexing Tests ===")

# --- Setup Data ---
# 3 Series, each length 4
# Series A: Labels "A"
# Series B: Labels "B" 
# Series C: Labels "C"

def make_series(label, length=4):
    mats = [MatrixInstance(np.eye(2)*i, label=label, tags={"idx": i}) for i in range(length)]
    return MatrixSeries(matrices=mats, label=label)

frame = MatrixFrame(series=[
    make_series("A"),
    make_series("B"),
    make_series("C")
])
print(f"Frame created: {frame.description}")

# --- 1. Row Access (NumPy Style) ---
print("\n--- 1. Row Access (frame[int]) ---")
# frame[2] should return a MatrixFrame of height 1 (Row 2 across A, B, C)
row_2 = frame[2]
assert isinstance(row_2, MatrixFrame), "frame[int] must return MatrixFrame (Row)"
assert len(row_2.labels) == 3, "Row should contain all 3 series"
assert len(row_2.series[0].matrices) == 1, "Row series should have length 1"
# Verify content
assert row_2["A"][0].matrix[0,0] == 2.0  # Matrix at index 2 was scaled by i=2
print("PASS: frame[int] -> Row Frame")


# --- 2. Column Access (Pandas Style) ---
print("\n--- 2. Column Access (frame[str]) ---")
col_b = frame["B"]
assert isinstance(col_b, MatrixSeries), "frame[str] must return MatrixSeries"
assert col_b.label == "B", "Returned wrong series"
assert len(col_b.matrices) == 4, "Series should be full length"
print("PASS: frame[str] -> Series")


# --- 3. 2D Access (NumPy Style) ---
print("\n--- 3. 2D Access (frame[row, col]) ---")

# A. Single Element [int, int]
# Row 3, Col 1 (Series B)
elem = frame[3, 1]
assert isinstance(elem, MatrixInstance), "frame[int, int] must return MatrixInstance"
assert elem.label == "B"
assert elem.matrix[0,0] == 3.0
print("PASS: frame[int, int] -> Instance")

# B. Single Element [int, str]
# Row 3, Series "C"
elem_c = frame[3, "C"]
assert isinstance(elem_c, MatrixInstance), "frame[int, str] must return MatrixInstance"
assert elem_c.label == "C"
print("PASS: frame[int, str] -> Instance")

# C. Column Slice [:, int]
# All Rows, Col 0 (Series A)
col_0 = frame[:, 0]
assert isinstance(col_0, MatrixSeries), "frame[:, int] must return MatrixSeries"
assert col_0.label == "A"
assert len(col_0.matrices) == 4
print("PASS: frame[:, int] -> Series")

# D. Row Slice [int, :] 
# Row 1, All Cols
row_1_all = frame[1, :]
assert isinstance(row_1_all, MatrixFrame), "frame[int, :] must return MatrixFrame (Row)"
assert len(row_1_all.labels) == 3
print("PASS: frame[int, :] -> Row Frame")

# E. Sub-Grid [slice, slice]
# Rows 0-2, Cols 1-3 (B, C)
sub_grid = frame[0:2, 1:3]
assert isinstance(sub_grid, MatrixFrame), "frame[slice, slice] must return MatrixFrame"
assert sub_grid.labels == ["B", "C"], f"Got {sub_grid.labels}"
assert len(sub_grid.series[0].matrices) == 2, "Series length should be 2"
print("PASS: frame[slice, slice] -> Sub-Grid Frame")

# F. List Selection [slice, list]
# All rows, Cols ["A", "C"]
subset = frame[:, ["A", "C"]]
assert isinstance(subset, MatrixFrame)
assert subset.labels == ["A", "C"]
print("PASS: frame[:, list] -> Subset Frame")

print("\n=== All Indexing Tests Passed! ===")

=== MatrixFrame Hybrid Indexing Tests ===
Frame created: matrixframe_3s_4i

--- 1. Row Access (frame[int]) ---
PASS: frame[int] -> Row Frame

--- 2. Column Access (frame[str]) ---
PASS: frame[str] -> Series

--- 3. 2D Access (frame[row, col]) ---
PASS: frame[int, int] -> Instance
PASS: frame[int, str] -> Instance
PASS: frame[:, int] -> Series
PASS: frame[int, :] -> Row Frame
PASS: frame[slice, slice] -> Sub-Grid Frame
PASS: frame[:, list] -> Subset Frame

=== All Indexing Tests Passed! ===
